In [3]:
# !pip install pyportfolioopt
# !pip install pulp
# !pip show cvxpy

In [1]:
import pandas as pd
import numpy as np
import requests


from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from pypfopt.efficient_frontier import EfficientFrontier

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [2]:
# Start with PyPortfolioOpt

# Read master dataset
df = pd.read_excel("master.xlsx")
if 'HDFCLIFE' in list(df.columns):
    del df['HDFCLIFE']

# Set Date as index and drop Date column
df = df.set_index(pd.DatetimeIndex(df['Date'].values))
df.drop(columns=['Date'], axis=1, inplace=True)

In [3]:
# Get different indices below
# https://www.niftyindices.com/indices/equity/broad-based-indices/

nifty_50_list = pd.read_csv('ind_nifty50list.csv')['Symbol']
nifty_100_list = pd.read_csv('ind_nifty100list.csv')['Symbol']
nifty_200_list = pd.read_csv('ind_nifty200list.csv')['Symbol']
nifty_500_list = pd.read_csv('ind_nifty500list.csv')['Symbol']
nifty_500_Multicap_502525 = pd.read_csv('ind_nifty500Multicap502525_list.csv')['Symbol']
nifty_midcap_50_list = pd.read_csv('ind_niftymidcap50list.csv')['Symbol']
nifty_midcap_100_list = pd.read_csv('ind_niftymidcap100list.csv')['Symbol']
nifty_smallcap_50_list = pd.read_csv('ind_niftysmallcap50list.csv')['Symbol']
nifty_smallcap_100_list = pd.read_csv('ind_niftysmallcap100list.csv')['Symbol']
nifty_smallcap_250_list = pd.read_csv('ind_niftysmallcap250list.csv')['Symbol']
nifty_largemidcap_250_list = pd.read_csv('ind_niftylargemidcap250list.csv')['Symbol']
nifty_midsmallcap_400_list = pd.read_csv('ind_niftymidsmallcap400list.csv')['Symbol']
nifty_tata = pd.read_csv('ind_nifty_tatalist.csv')['Symbol']
nifty_finance = pd.read_csv('ind_niftyfinancelist.csv')['Symbol']
nifty_fmcg = pd.read_csv('ind_niftyfmcglist.csv')['Symbol']
nifty_it = pd.read_csv('ind_niftyitlist.csv')['Symbol']
nifty_metal = pd.read_csv('ind_niftymetallist.csv')['Symbol']
nifty_pharma = pd.read_csv('ind_niftypharmalist.csv')['Symbol']
nifty_energy = pd.read_csv('ind_niftyenergylist.csv')['Symbol']

In [4]:
# Create a dictonary of indices with name of the index and all the stocks in that index

indices = {'NIFTY_50':nifty_50_list,
           'NIFTY_100':nifty_100_list,
           'NIFTY_200':nifty_200_list, 
           'NIFTY_500':nifty_500_list,
           'NIFTY_500_MULTICAP_502525':nifty_500_Multicap_502525,
           'NIFTY_50_MIDCAP':nifty_midcap_50_list,
           'NIFTY_100_MIDCAP':nifty_midcap_100_list,
           'NIFTY_50_SMALLCAP':nifty_smallcap_50_list,
           'NIFTY_100_SMALLCAP':nifty_smallcap_100_list,
           'NIFTY_250_SMALLCAP':nifty_smallcap_250_list,
           'NIFTY_250_LARGE_MIDCAP':nifty_largemidcap_250_list,
           'NIFTY_400_MID_SMALLCAP':nifty_midsmallcap_400_list,
           'NIFTY_METAL':nifty_metal,
           'NIFTY_TATA':nifty_tata,
           'NIFTY_FINANCE':nifty_finance,
           'NIFTY_FMCG':nifty_fmcg,
           'NIFTY_IT':nifty_it,
           'NIFTY_PHARMA':nifty_pharma,
           'NIFTY_ENERGY':nifty_energy}

In [1]:
# Run for all the indices at once

for key, index in indices.items():

    df1 = df[df.columns[df.columns.isin(index)]]
    assets = df1.columns
    mu = expected_returns.mean_historical_return(df1)
    S = risk_models.sample_cov(df1) 
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(key)

    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    portfolio_val = 3000
    latest_prices = get_latest_prices(df1)
    weights = cleaned_weights
    da = DiscreteAllocation(weights, latest_prices,
                            total_portfolio_value=portfolio_val)
    allocation, leftover = da.lp_portfolio()
    print('Discrete allocation : ', allocation)
    print('Funds remaining : ', leftover)
    print('Amount to invest : ' + str(portfolio_val))
    print('\n')